<h1>Import Required Libraries</h1>

In [11]:
import yfinance as yf
import pickle
import numpy as np
import pandas as pd


<h1>Select the Stock</h1>

In [12]:
ticker = "RELIANCE.NS"
print(f"\n Predicting signal for {ticker}")



 Predicting signal for RELIANCE.NS


<h1>Load Saved Model Files</h1>

In [14]:
with open(f"{ticker}_model.pkl", "rb") as f:
    model = pickle.load(f)

# Load scaler
with open(f"{ticker}_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# Load feature columns
with open(f"{ticker}_features.pkl", "rb") as f:
    feature_columns = pickle.load(f)

<h1>Fetch Recent Market Data</h1>

In [15]:
ticker = "RELIANCE.NS"
data = yf.download(ticker, period="3mo", auto_adjust=True)

if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)
    

[*********************100%***********************]  1 of 1 completed


<h1>Feature Engineering</h1>

In [17]:
data["Return"] = data["Close"].pct_change()

data["SMA_5"] = data["Close"].rolling(5).mean()
data["SMA_20"] = data["Close"].rolling(20).mean()
data["EMA_10"] = data["Close"].ewm(span=10).mean()

data["Volatility"] = data["Return"].rolling(10).std()
data["Momentum"] = data["Close"] - data["Close"].shift(5)

for i in range(1,6):
    data[f"Lag_{i}"] = data["Return"].shift(i)

data.dropna(inplace=True)

<h1>Prepare Latest Feature Row</h1>

In [27]:
X_live = data[feature_columns].iloc[-1:]
X_live_scaled = scaler.transform(X_live)

<h1>Predict Signal</h1>

In [40]:
prediction = model.predict(X_live_scaled)[0]
signal = model.predict_proba(X_live_scaled)[0][1]

<h1>Print Human-Readable Output</h1>

In [41]:
if signal > 0.65:
    signal = "BUY 📈"
elif signal < 0.35:
    signal = "SELL 📉"
else:
    signal = "HOLD ⏳"

print("Signal:", signal)


Signal: HOLD ⏳


<h1>Save the Stock data in dataset </h1>

In [42]:
data.to_csv("signal.csv")